<a href="https://colab.research.google.com/github/Abishekdevs/trinity/blob/main/work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps
!pip install -q -U google-generativeai
!pip install --upgrade algoliasearch

In [ ]:
#@title 📍 AI Restaurant Explorer (V3 - Stable) { run: "auto" }
LOCATION_INPUT = "Austin, TX, USA" #@param {type:"string"}

import requests, json
from google.colab import userdata

def debug_geocode():
    key = userdata.get('MAPS_API_KEY')
    # Using a slightly different URL format for better compatibility
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": LOCATION_INPUT,
        "key": key
    }

    print(f"--- Debugging Connection ---")
    try:
        response = requests.get(url, params=params)
        data = response.json()

        status = data.get("status")
        error_msg = data.get("error_message", "No error message provided.")

        if status == "OK":
            results = data["results"][0]["geometry"]["location"]
            print(f"✅ Success! Found: {results}")
            return results
        else:
            print(f"❌ Status: {status}")
            print(f"❌ Details: {error_msg}")

            if status == "REQUEST_DENIED":
                print("\n💡 ACTION REQUIRED:")
                print("1. Ensure 'Geocoding API' is ENABLED in Cloud Console.")
                print("2. Ensure 'Billing' is linked to this specific project.")
                print("3. Ensure 'Application Restrictions' is set to 'None' in API Key settings.")
    except Exception as e:
        print(f"❌ Critical Error: {e}")

debug_geocode()

In [ ]:
#@title 📍 AI Advanced Market Intelligence Explorer (v5 - Max Results & Split Address)
LOCATION_INPUT = "Madurai, tamil Nadu, India" #@param {type:"string"}
RADIUS_METERS = 10000 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "Updated_Advanced_Market_Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)

    if not ws_rest.get_all_values():
        ws_rest.append_row([
            'timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country',
            'cuisine', 'website', 'unique_selling_point', 'market_gap_opportunity', 'social_links'
        ])

    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)

    if not ws_menu.get_all_values():
        ws_menu.append_row([
            'id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'
        ])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types:
            city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types:
            state = c.get('shortText', 'N/A')
        elif 'country' in types:
            country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "1. Identify CUISINE, WEBSITE, and SOCIAL LINKS.\n"
        "2. Identify its UNIQUE SELLING POINT (USP).\n"
        "3. MARKET GAP: What is missing compared to local rivals?\n"
        "4. Provide a MENU of at least 20 items.\n"
        "5. Flag if items are 'Unique' and why.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_app():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)
    existing_rows = ws_rest.get_all_values()
    existing_entries = {(row[2], row[4]) for row in existing_rows[1:]}

    res_geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if res_geo['status'] != 'OK': return print("❌ Location not found.")
    lat, lng = res_geo['results'][0]['geometry']['location'].values()

    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    # UPDATED: Increased maxResultCount to 20 (API Maximum)
    data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 20,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
    }
    nearby_data = requests.post(url, headers=headers, json=data).json().get('places', [])

    unique_nearby = [p for p in nearby_data if (p['displayName']['text'], p['formattedAddress']) not in existing_entries]

    if not unique_nearby:
        return print(f"✅ No new restaurants found. (Skipped {len(nearby_data)} existing entries)")

    print(f"🔍 Processing {len(unique_nearby)} new restaurants...")
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in unique_nearby:
        name = p['displayName']['text']
        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"📊 Auditing: {name} in {city}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})

        ws_rest.append_row([
            curr_time, rest_uuid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'), d.get('market_gap'),
            ", ".join(d.get('social_links', []))
        ])

        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in ai.get('menu', [])]
        if menu_rows: ws_menu.append_rows(menu_rows)

        time.sleep(2)

    print(f"\n✨ Complete! Check '{WORKBOOK_NAME}'.")

run_app()

In [ ]:
#@title 📍 AI Batch Restaurant Audit (v7)
RESTAURANT_NAMES = "Anjappar Chettinad, Madurai Kumar Mess, Karaikudi Mess, Geetham Veg Restaurant" #@param {type:"string"}
CITY_LOCATION = "chennai, Tamil Nadu, India" #@param {type:"string"}
WORKBOOK_NAME = "Updated_Advanced_Market_Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'unique_selling_point', 'market_gap_opportunity', 'social_links'])

    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Analyze the restaurant: '{restaurant_name}' at '{location}'.\n"
        "1. Identify its CUISINE, WEBSITE, and SOCIAL LINKS.\n"
        "2. Identify its UNIQUE SELLING POINT (USP).\n"
        "3. MARKET GAP: What is missing compared to local rivals?\n"
        "4. Provide a MENU (at least 20 items).\n"
        "5. For each item, state if it's 'Unique' and give reasoning.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_batch_audit():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)

    # Pre-fetch existing entries for deduplication
    existing_rows = ws_rest.get_all_values()
    existing_fingerprints = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # Split input into a list
    names_to_process = [n.strip() for n in RESTAURANT_NAMES.split(",") if n.strip()]
    print(f"🚀 Starting Batch Audit for {len(names_to_process)} restaurants...\n")

    for raw_name in names_to_process:
        search_query = f"{raw_name} in {CITY_LOCATION}"
        print(f"🔍 Searching: {search_query}...")

        url = "https://places.googleapis.com/v1/places:searchText"
        headers = {
            "Content-Type": "application/json",
            "X-Goog-Api-Key": MAPS_API_KEY,
            "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
        }
        data = {"textQuery": search_query, "maxResultCount": 1}

        try:
            resp = requests.post(url, headers=headers, json=data).json()
            places = resp.get('places', [])
            if not places:
                print(f"⚠️  Not found: '{raw_name}'. Skipping.")
                continue

            target = places[0]
            name, loc = target['displayName']['text'], target['formattedAddress']

            # Dedup Check
            if (name.lower(), loc.lower()) in existing_fingerprints:
                print(f"⏩ Skipped: '{name}' already exists.")
                continue

            # Process with AI
            ai = enrich_market_intelligence(name, loc)
            if not ai:
                print(f"❌ AI Error for '{name}'. Skipping.")
                continue

            # Save Data
            phone = target.get('internationalPhoneNumber', 'N/A')
            city, state, country = parse_address_components(target.get('addressComponents', []))
            rest_uuid = str(uuid.uuid4())
            d = ai.get('details', {})

            ws_rest.append_row([
                time.strftime("%Y-%m-%d %H:%M:%S"), rest_uuid, name, phone, loc,
                city, state, country, d.get('cuisine'), d.get('website'),
                d.get('usp'), d.get('market_gap'), ", ".join(d.get('social_links', []))
            ])

            menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in ai.get('menu', [])]
            if menu_rows: ws_menu.append_rows(menu_rows)

            print(f"✅ Success: '{name}' processed with {len(menu_rows)} menu items.")
            time.sleep(2) # Protect API quota

        except Exception as e:
            print(f"⚠️ Unexpected error with '{raw_name}': {e}")

    print(f"\n✨ Batch processing complete!")

run_batch_audit()

In [ ]:
#@title 🥘 AI Dish-Based Restaurant Search (v8)
DISH_NAME = "Mutton biriyani" #@param {type:"string"}
CITY_LOCATION = "chennai, Tamil Nadu, India" #@param {type:"string"}
WORKBOOK_NAME = "Updated_Advanced_Market_Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    # Setup Restaurant Sheet
    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'unique_selling_point', 'market_gap_opportunity', 'social_links'])

    # Setup Menu Sheet
    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Analyze the restaurant: '{restaurant_name}' at '{location}'.\n"
        "1. Identify its CUISINE, WEBSITE, and SOCIAL LINKS.\n"
        "2. Identify its UNIQUE SELLING POINT (USP).\n"
        "3. MARKET GAP: What is this restaurant missing compared to other popular spots in the same city?\n"
        "4. Provide a MENU (at least 20 items).\n"
        "5. For each item, state if it's 'Unique' and give reasoning.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_dish_audit():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)

    # 1. Fetch existing for dedup
    existing_rows = ws_rest.get_all_values()
    existing_fingerprints = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # 2. Search for the DISH
    search_query = f"Best {DISH_NAME} in {CITY_LOCATION}"
    print(f"🥘 Searching for the best '{DISH_NAME}' in {CITY_LOCATION}...")

    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    # Getting top 5 restaurants for this dish
    data = {"textQuery": search_query, "maxResultCount": 5}

    resp = requests.post(url, headers=headers, json=data).json()
    places = resp.get('places', [])

    if not places:
        return print(f"❌ No restaurants found for '{DISH_NAME}'.")

    print(f"🔍 Found {len(places)} top candidates for this dish. Checking for duplicates...")

    for p in places:
        name, loc = p['displayName']['text'], p['formattedAddress']

        # Dedup Check
        if (name.lower(), loc.lower()) in existing_fingerprints:
            print(f"⏩ Skipped: '{name}' already exists in your database.")
            continue

        print(f"📊 Auditing: {name}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        # Save Logic
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))
        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})

        ws_rest.append_row([
            time.strftime("%Y-%m-%d %H:%M:%S"), rest_uuid, name, phone, loc,
            city, state, country, d.get('cuisine'), d.get('website'),
            d.get('usp'), d.get('market_gap'), ", ".join(d.get('social_links', []))
        ])

        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in ai.get('menu', [])]
        if menu_rows: ws_menu.append_rows(menu_rows)

        print(f"✅ Success: '{name}' added.")
        time.sleep(2)

    print(f"\n✨ Dish-based search complete!")

run_dish_audit()

In [ ]:
# 1. INSTALL LIBRARIES (Run this first)

# 2. THE COMPLETE SCRIPT
#@title 📍 AI Advanced Market Intelligence Explorer (v18)
LOCATION_INPUT = "Madurai, tamil Nadu, India" #@param {type:"string"}
RADIUS_METERS = 10000 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "Updated Advanced Market Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread
# ADDED: Import Algolia v4 Search Client
from algoliasearch.search.client import SearchClient

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Configure API Keys from Secrets
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')

    # ADDED: Initialize Algolia Search Client v4
    # Note: Ensure you have ALGOLIA_APP_ID and ALGOLIA_API_KEY in your Secrets
    algolia_client = SearchClient(
        userdata.get('ALGOLIA_APP_ID'),
        userdata.get('ALGOLIA_API_KEY')
    )
    ALG_INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    # Select Gemini Model
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')

    print("✅ All services (Gemini, Maps, Algolia) Initialized Successfully.")
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])

    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_app():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)
    existing_rows = ws_rest.get_all_values()
    existing_entries = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # 1. Geocoding
    res_geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if res_geo['status'] != 'OK': return print("❌ Geocoding Error.")
    lat, lng = res_geo['results'][0]['geometry']['location'].values()

    # 2. Nearby Search
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 20,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
    }
    nearby_data = requests.post(url, headers=headers, json=data).json().get('places', [])

    # Filter out duplicates
    unique_nearby = [p for p in nearby_data if (p['displayName']['text'].lower(), p['formattedAddress'].lower()) not in existing_entries]
    if not unique_nearby:
        return print(f"✅ No new restaurants found. (Skipped {len(nearby_data)} existing entries)")

    print(f"🔍 Processing {len(unique_nearby)} new restaurants...")
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in unique_nearby:
        name = p['displayName']['text']
        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"📊 Auditing: {name} in {city}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})

        # 3. SAVE TO GOOGLE SHEETS
        ws_rest.append_row([
            curr_time, rest_uuid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'), d.get('market_gap'),
            ", ".join(d.get('social_links', []))
        ])

        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in ai.get('menu', [])]
        if menu_rows: ws_menu.append_rows(menu_rows)

        # 4. ADDED: SYNC TO ALGOLIA
        try:
            algolia_record = {
                "objectID": rest_uuid, # Use the same UUID for deduplication
                "name": name,
                "location": loc,
                "city": city,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap')
            }
            # Algolia v4 use client.save_object
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body=algolia_record)
            print(f"   ✅ Synced to Algolia: {name}")
        except Exception as alg_e:
            print(f"   ⚠️ Algolia Sync Failed: {alg_e}")

        time.sleep(2) # Prevent API rate limiting

    print(f"\n✨ Batch Complete! Added {len(unique_nearby)} new records to Sheets and Algolia.")

run_app()

In [4]:
from algoliasearch.search.client import SearchClientSync # Use Sync version
from google.colab import userdata
import uuid

try:
    APP_ID = userdata.get('ALGOLIA_APP_ID')
    API_KEY = userdata.get('ALGOLIA_API_KEY')
    INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    # Use SearchClientSync to avoid "coroutine never awaited" errors
    client = SearchClientSync(APP_ID, API_KEY)

    dummy_record = {
        "objectID": f"test_{uuid.uuid4().hex[:5]}",
        "name": "Sync Debug Test",
        "status": "Success"
    }

    # This will now execute immediately
    client.save_object(index_name=INDEX_NAME, body=dummy_record)
    print(f"✅ Success! Record {dummy_record['objectID']} is now in Algolia.")

except Exception as e:
    print(f"❌ Error: {e}")

✅ Success! Record test_b9817 is now in Algolia.


In [ ]:
# 1. INSTALL LIBRARIES (Required for first run)
!pip install --upgrade -q algoliasearch gspread google-auth

#@title 📍 AI Advanced Market Intelligence Explorer (v18)
LOCATION_INPUT = "Theni, Tamil Nadu, India" #@param {type:"string"}
RADIUS_METERS = 15500 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "Updated Advanced Market Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread

# UPDATED: Import the Synchronous version of the Algolia client
from algoliasearch.search.client import SearchClientSync

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Configure Gemini and Maps
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')

    # UPDATED: Initialize Algolia SearchClientSync
    # Ensure you have ALGOLIA_APP_ID, ALGOLIA_API_KEY, and ALGOLIA_INDEX_NAME in Secrets
    algolia_client = SearchClientSync(
        userdata.get('ALGOLIA_APP_ID'),
        userdata.get('ALGOLIA_API_KEY')
    )
    ALG_INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    # Select Gemini Model
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')

    print("✅ Services (Gemini, Maps, Algolia Sync) Initialized Successfully.")
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    # (Existing logic to open/create 'Restaurants' and 'Menus' worksheets)
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])

    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    # (Existing logic to extract city, state, country)
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    # (Existing AI analysis logic)
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_app():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)
    existing_rows = ws_rest.get_all_values()
    existing_entries = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # 1. Geocoding
    res_geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if res_geo['status'] != 'OK': return print("❌ Geocoding Error.")
    lat, lng = res_geo['results'][0]['geometry']['location'].values()

    # 2. Nearby Search (Places API v1)
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 20,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
    }
    nearby_data = requests.post(url, headers=headers, json=data).json().get('places', [])

    # Filter out duplicates (Page 4 of code.pdf)
    unique_nearby = [p for p in nearby_data if (p['displayName']['text'].lower(), p['formattedAddress'].lower()) not in existing_entries]
    if not unique_nearby:
        return print(f"✅ No new restaurants found.")

    print(f"🔍 Auditing {len(unique_nearby)} new restaurants...")
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in unique_nearby:
        name = p['displayName']['text']
        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"📊 Analyzing: {name}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})

        # 3. SAVE TO GOOGLE SHEETS
        ws_rest.append_row([
            curr_time, rest_uuid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'), d.get('market_gap'),
            ", ".join(d.get('social_links', []))
        ])

        # 4. ADDED: SYNC TO ALGOLIA (Using Synchronous save_object)
        try:
            algolia_record = {
                "objectID": rest_uuid, # Consistent ID for deduplication
                "name": name,
                "location": loc,
                "city": city,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap')
            }
            # Standard function call (no await needed with SearchClientSync)
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body=algolia_record)
            print(f"   ✅ Data synced to Algolia.")
        except Exception as alg_e:
            print(f"   ⚠️ Algolia sync error: {alg_e}")

        # Save menu items to sheet
        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in ai.get('menu', [])]
        if menu_rows: ws_menu.append_rows(menu_rows)

        time.sleep(2) # Avoid rate limits

    print(f"\n✨ Batch Complete!")

run_app()

In [ ]:
# 1. INSTALL LIBRARIES
!pip install --upgrade -q algoliasearch gspread google-auth

#@title 📍 AI Advanced Market Intelligence Explorer (v18)
LOCATION_INPUT = "Madurai, tamil Nadu, India" #@param {type:"string"}
RADIUS_METERS = 15500 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "Final Advanced Market Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread

# UPDATED: Import the Synchronous version of the Algolia client
from algoliasearch.search.client import SearchClientSync

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Configure Gemini and Maps
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')

    # UPDATED: Initialize Algolia SearchClientSync
    algolia_client = SearchClientSync(
        userdata.get('ALGOLIA_APP_ID'),
        userdata.get('ALGOLIA_API_KEY')
    )
    ALG_INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    # Select Gemini Model
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')

    print("✅ All services (Gemini, Maps, Algolia Sync) Initialized Successfully.")
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])

    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_app():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)
    existing_rows = ws_rest.get_all_values()
    existing_entries = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # 1. Geocoding
    res_geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if res_geo['status'] != 'OK': return print("❌ Geocoding Error.")
    lat, lng = res_geo['results'][0]['geometry']['location'].values()

    # 2. Nearby Search
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 20,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
    }
    nearby_data = requests.post(url, headers=headers, json=data).json().get('places', [])

    unique_nearby = [p for p in nearby_data if (p['displayName']['text'].lower(), p['formattedAddress'].lower()) not in existing_entries]
    if not unique_nearby:
        return print(f"✅ No new restaurants found.")

    print(f"🔍 Auditing {len(unique_nearby)} new restaurants...")
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in unique_nearby:
        name = p['displayName']['text']
        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"📊 Analyzing: {name}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})
        menu_items = ai.get('menu', []) # Extract menu data for dual-saving

        # 3. SAVE TO GOOGLE SHEETS
        ws_rest.append_row([
            curr_time, rest_uuid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'), d.get('market_gap'),
            ", ".join(d.get('social_links', []))
        ])

        # 4. SYNC TO ALGOLIA (Including Menu Data)
        try:
            algolia_record = {
                "objectID": rest_uuid,
                "name": name,
                "location": loc,
                "city": city,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap'),
                # ADDED: Include the menu as an array within the record
                "menu": menu_items
            }
            # Synchronous call (no await needed)
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body=algolia_record)
            print(f"   ✅ Data + Menu synced to Algolia.")
        except Exception as alg_e:
            print(f"   ⚠️ Algolia Sync Failed: {alg_e}")

        # 5. SAVE MENU ITEMS TO SHEETS (Existing logic)
        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in menu_items]
        if menu_rows: ws_menu.append_rows(menu_rows)

        time.sleep(2) # Avoid rate limits

    print(f"\n✨ Batch Complete!")

run_app()